In [15]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings('ignore')

############################################## PREPROCESSING ##############################################
from sklearn.preprocessing import MinMaxScaler

################################################# METRICS #################################################
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_curve

################################### MODEL SELECTION & OPTIMIZATION ########################################
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

######################################### DECISION TREES PLOTS ############################################
from sklearn.tree import export_graphviz
import graphviz
import pydotplus

In [4]:
df= pd.read_csv(r"C:\Users\pedro\OneDrive\Desktop\DSProject Folder\src\Financial Data 14-18\Int DFs\df_mod.csv")

In [5]:
df.shape

(19718, 26)

In [6]:
df.head(10)

,Effect of forex changes on cash,SG&A Expenses Growth,Receivables Turnover,5Y Revenue Growth (per Share),3Y Shareholders Equity Growth (per Share),Issuance (buybacks) of shares,eBITperRevenue,Net cash flow / Change in cash,priceSalesRatio,SG&A to Revenue,...,Operating Cash Flow,3Y Net Income Growth (per Share),returnOnCapitalEmployed,Tangible Book Value per Share,Investing Cash flow,Gross Margin,PFCF ratio,5Y Operating CF Growth (per Share),Class,Following Year Price Variation [%]
0,-1.564869e+07,1.73130,6.48354,0.06600,0.79730,1.767840e+06,0.050221,4.463169e+08,0.095727,0.0922,...,5.267456e+08,0.25206,0.0000,4.49390,-6.866936e+08,0.2487,1.3589,0.11996,0,-25.512193
1,0.000000e+00,0.02340,90.79370,0.10380,0.07890,-4.130000e+08,0.027578,1.630000e+08,0.000000,0.1545,...,3.573000e+09,0.18920,0.0859,25.72400,-4.771000e+09,0.2057,14.6302,0.09370,1,33.118297
2,0.000000e+00,-0.00600,27.17690,-0.02900,0.00000,3.321700e+07,0.026436,1.695400e+07,0.000000,0.2570,...,7.020460e+08,0.00000,0.1062,134.78500,-3.649240e+08,0.2869,17.2736,0.11640,1,2.752291
3,-2.920000e+07,-0.02200,12.22500,0.05670,0.02170,-1.637200e+09,0.168072,1.259000e+08,1.553911,0.1940,...,2.541000e+09,0.01770,0.1041,15.42900,-5.618000e+08,0.3557,17.6902,0.08280,1,12.897715
4,-3.760000e+08,0.01610,20.39100,0.09610,0.00000,-3.833000e+09,0.145332,-4.720000e+08,1.299472,0.0874,...,7.739000e+09,-0.00840,0.3752,15.32700,-9.960000e+08,0.2413,19.2150,0.03770,1,13.980937
5,-2.230000e+08,-0.02560,6.74490,-0.04940,-0.06070,-1.700000e+09,0.094177,-9.910000e+08,1.644111,0.2470,...,3.562000e+09,-0.13540,0.0376,13.71900,-1.642000e+09,0.3679,31.6718,-0.09250,1,23.809818
6,0.000000e+00,0.08500,205.20500,0.01770,-0.05190,-9.390000e+08,0.349971,1.460000e+08,3.319691,0.1035,...,4.663000e+09,0.15880,0.1948,8.67300,1.770000e+08,0.4143,21.6402,0.07260,1,23.865489
7,3.788960e+07,-0.00760,11.93360,0.15340,0.09406,3.131806e+07,0.408287,-7.570785e+08,2.252915,0.2883,...,7.052460e+09,0.05698,0.2515,0.98010,-1.790462e+09,0.6635,6.7470,0.19388,0,-22.605312
8,6.943980e+05,0.34600,0.00000,0.45210,0.12170,0.000000e+00,0.214357,8.484990e+07,0.000000,0.3379,...,1.015584e+08,0.28460,0.0000,0.88800,-1.740287e+07,0.5172,6.8263,0.27070,1,65.373665
9,-1.320000e+08,-0.01442,15.78346,0.03482,0.07494,0.000000e+00,0.134957,-1.610000e+08,12.993151,0.1565,...,2.140000e+09,0.03054,0.0433,20.10842,-3.490000e+08,0.3000,22.0915,0.04848,1,2.065963


Apply Train Test Split so we save a portion of unseen data to test the model

In [8]:
X = df.drop(columns=['Following Year Price Variation [%]', 'Class'])
y = df['Class']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42, stratify=y)

In [10]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((15774, 24), (3944, 24), (15774,), (3944,))

In [13]:
def apply_model (X, y, model):

    score_train = []
    score_val = []

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=99) # Because it's a classification problem. In the regression problem we will use KFold
    
    pipe = Pipeline([
        ('scaler', MinMaxScaler()),  #Scale the data
        ('clf', model) #Model is passed as an argument (KNN, Decision Tree, MLP, etc)
    ])
    #Cross Validation

    for train_idx, val_idx in skf.split(X, y):
        X_t, X_v = X.iloc[train_idx], X.iloc[val_idx]
        y_t, y_v = y.iloc[train_idx], y.iloc[val_idx]

        pipe.fit(X_t, y_t)

        pred_train = pipe.predict(X_t)
        pred_val = pipe.predict(X_v)

        score_train.append(f1_score(y_t, pred_train))  #Using F1-score as a metric because its a classification problem
        score_val.append(f1_score(y_v, pred_val))
    
    avg_train = round(np.mean(score_train), 3)
    avg_val = round(np.mean(score_val), 3)
    std_train = round(np.std(score_train), 2)
    std_val = round(np.std(score_val), 2) 

    return avg_train, std_train, avg_val, std_val

In [14]:
def model_assessment(X, y , **models):

    results = {}
    
    for name, model in models.items():
        avg_train, std_train, avg_val, std_val = apply_model(X, y, model)
        results[name] = {
            "Train": f"{avg_train:.3f} +/- {std_train:.2f}",
            "Validation": f"{avg_val:.3f} +/- {std_val:.2f}"
        }
    return pd.DataFrame(results).T

In [16]:
################################################# MODELS ##################################################
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

Create the dictionary of models to evaluate

In [17]:
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000, solver='lbfgs', random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "Neural Network": MLPClassifier(hidden_layer_sizes=(50,50), max_iter=500, random_state=42)
}

In [18]:
results_df = model_assessment(X_train, y_train, **models)
print(results_df)


                              Train      Validation
Random Forest        1.000 +/- 0.00  0.679 +/- 0.00
Gradient Boosting    0.729 +/- 0.00  0.691 +/- 0.00
Logistic Regression  0.695 +/- 0.00  0.692 +/- 0.00
KNN                  0.750 +/- 0.00  0.609 +/- 0.00
Neural Network       0.687 +/- 0.02  0.683 +/- 0.01


Random Forest sems to be highly overfiited with a perfect score in the training phase and a considerable decrease in the Valiation Phase  
Best Models seem to be Gradient Boosting an Logistic Regression. Not overfiited and with the best Validation Scores.  

These 2 will move on to Fine Tuning/Model Optimization


In [20]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from scipy.stats import uniform, randint

Let's use RandomizedSearchCV as GridSearch tests all possible combinations and is very computationally expensive

In [22]:
gb = GradientBoostingClassifier(random_state=42)
lr = LogisticRegression(max_iter=500, random_state=42)

Gradient Boosting (Not sensitive to scalling, tree based)

In [23]:
param_dist_gb = {
    'n_estimators': randint(50, 300),
    'learning_rate': uniform(0.01, 0.2),
    'max_depth': randint(3, 10)
}

random_search_gb = RandomizedSearchCV(
    estimator=gb,
    param_distributions=param_dist_gb,
    n_iter=50,     # number of random combinations to try
    cv=5,
    scoring='f1',
    n_jobs=-1,
    random_state=42
)

random_search_gb.fit(X_train, y_train)
print(random_search_gb.best_params_)
print(random_search_gb.best_score_)

{'learning_rate': np.float64(0.011413261043943482), 'max_depth': 3, 'n_estimators': 98}
0.7064103616384327


Logistic Regression

In [26]:
pipeline_tun = Pipeline([
    ('scaler', MinMaxScaler()),  #Scale the data
    ('classifier', lr)
])

param_dist_lr = {
    'classifier__C': uniform(0.01, 10),           # regularization strength
    'classifier__penalty': ['l1', 'l2'],          # type of regularization
    'classifier__solver': ['liblinear', 'saga']   # solvers that support l1/l2
}

random_search_lr = RandomizedSearchCV(
    estimator=pipeline_tun,
    param_distributions=param_dist_lr,
    n_iter=20,     # number of random combinations to try
    cv=5, #For each combination of hyperparameters, perform 5-fold cross-validation. Trains on 4 folds and validates on the remaining fold.
    scoring='f1', #RS will try to maximize the F1-score
    n_jobs=-1, #Controls parallel processing, -1 means using all processors, 1= single processor etc
    random_state=42
)

random_search_lr.fit(X_train, y_train)
print(random_search_lr.best_params_)
print(random_search_lr.best_score_)

{'classifier__C': np.float64(0.21584494295802448), 'classifier__penalty': 'l2', 'classifier__solver': 'saga'}
0.6960541802339144


In [27]:
final_model_gb = GradientBoostingClassifier(learning_rate=0.011413261043943482, 
                                            max_depth= 3,
                                            n_estimators= 98,
                                            random_state=42)

In [29]:
final_model_lr=LogisticRegression(C=0.21584494295802448,
                                    penalty= 'l2', 
                                    solver= 'saga',
                                    max_iter=500,  #Not fine tuned but default =100 and LR oftern needs more iterations to converge
                                    random_state=42)

In [30]:
models_final = {"Gradient Boosting": final_model_gb,
                "Logistic Regression": final_model_lr}

In [31]:
fresults_df= model_assessment(X_train, y_train, **models_final)
print(fresults_df)

                              Train      Validation
Gradient Boosting    0.713 +/- 0.00  0.707 +/- 0.00
Logistic Regression  0.696 +/- 0.00  0.695 +/- 0.00


GB : Very small gap, model generalizes well, there is no overfitting.  

LR : Nearly identical. Excellent stability, just slightly less predictive power than Gradient Boosting.

Save/Export Models

In [32]:
import joblib

In [ ]:
joblib.dump(final_model_gb, "final_model_gb.pkl")
joblib.dump(final_model_lr, "final_model_lr.pkl")

['final_model_lr.pkl']

In [36]:
scaler = MinMaxScaler().fit(X_train)
joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']

In [35]:
joblib.dump((X_test, y_test), "test_data.pkl")

['test_data.pkl']